In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8484.40it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4813.45it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2738016.34it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8982.45it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4811.44it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 92625.66it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8548.65it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4881.14it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 75038.89it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 9137.92it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4848.27it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 92696.25it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.DUET()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['lm_filters'] = 32
model.params['lm_hidden_sizes'] = [32]
model.params['dm_filters'] = 32
model.params['dm_kernel_size'] = 3
model.params['dm_d_mpool'] = 4
model.params['dm_hidden_sizes'] = [32]
model.params['dropout_rate'] = 0.5
model.params['optimizer'] = 'adagrad'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DUET.
Parameter "embedding_trainable" set to True.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5002200     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (N

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))

In [10]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [11]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 7s 68ms/step - loss: 1.1611
Validation: normalized_discounted_cumulative_gain@3(0): 0.5285485508994274 - normalized_discounted_cumulative_gain@5(0): 0.6002174150475211 - mean_average_precision(0): 0.5483873824220945
Epoch 2/30
102/102 [==============================] - 6s 54ms/step - loss: 0.7425
Validation: normalized_discounted_cumulative_gain@3(0): 0.5539639184214037 - normalized_discounted_cumulative_gain@5(0): 0.6120134302500022 - mean_average_precision(0): 0.5739963779648841
Epoch 3/30
102/102 [==============================] - 6s 60ms/step - loss: 0.6449

Epoch 3/30
Validation: normalized_discounted_cumulative_gain@3(0): 0.5754291916974498 - normalized_discounted_cumulative_gain@5(0): 0.6458393893065979 - mean_average_precision(0): 0.5998328780890881
Epoch 4/30
102/102 [==============================] - 6s 61ms/step - loss: 0.5448
Validation: normalized_discounted_cumulative_gain@3(0): 0.5993020241109742 - normalized_discount

Validation: normalized_discounted_cumulative_gain@3(0): 0.5858505248511132 - normalized_discounted_cumulative_gain@5(0): 0.6448034790882025 - mean_average_precision(0): 0.6012140002360248
Epoch 24/30
102/102 [==============================] - 6s 60ms/step - loss: 0.0708

Epoch 24/30
Validation: normalized_discounted_cumulative_gain@3(0): 0.5828995773354282 - normalized_discounted_cumulative_gain@5(0): 0.6380952646106933 - mean_average_precision(0): 0.5924163408420962
Epoch 25/30
102/102 [==============================] - 6s 60ms/step - loss: 0.0596
Validation: normalized_discounted_cumulative_gain@3(0): 0.5828995773354282 - normalized_discounted_cumulative_gain@5(0): 0.6380952646106933 - mean_average_precision(0): 0.5924163408420962
Epoch 25/30
Validation: normalized_discounted_cumulative_gain@3(0): 0.5907564536681598 - normalized_discounted_cumulative_gain@5(0): 0.6457727504590735 - mean_average_precision(0): 0.6027660580655791
Epoch 26/30
102/102 [==============================] - 6s